In [ ]:

#ets
if(True):

    #take last nyse date and extrapolate weeks from that point
    #starting from next week
    newDates = pd.date_range((subset.index[-1] + dt.timedelta(7)).strftime('%Y-%m-%d'), (subset.index[-1] + dt.timedelta(92)).strftime('%Y-%m-%d'), freq='W-'+nyse_dates.index[-1].strftime('%a')).map(lambda t: t.strftime('%Y-%m-%d'))

    print(subset.index[-1].strftime('%Y-%m-%d'))
    print(newDates[0])
    #data = subset[['Adj Close']].asfreq('D').interpolate().asfreq('W-'+subset.index[-1].strftime('%a'))
    data = subset[['Adj Close']].asfreq('D').interpolate().asfreq('W-'+nyse_dates.index[-1].strftime('%a'))            

    #if(np.where(np.array(list__)==s)[0][0]%batchClearLimit==0):        
    #client.restart()

    cv_inner = TimeSeriesSplit(n_splits=11,test_size=13)

    #has to be different makeup than cv_inner because there is no randomization.  Recommend a number that isn't a divisor of cv_inner's n_splits to ensure training is mixed up and not aligned
    cv_outer = TimeSeriesSplit(n_splits=10,test_size=13)

    outer_results = []

    #sm.tsa.statespace.ExponentialSmoothing
    """
    flags = []
    seasonal_flags1 = ['add','mul',None]

    trend_flags2 = ['add','mul',None]

    damped__trend_flags3 = [True,False]

    for sf in seasonal_flags1:
        for dt_ in trend_flags2:
            if(dt_ is None):          
                for d_ in damped__trend_flags3:
                    flags.append([sf,dt_,d_])
    """


    flags = [['add', 'add', True],\
     ['add', 'add', False],\
     ['add', 'mul', True],\
     ['add', 'mul', False],\
     ['add', None],\
     ['mul', 'add', True],\
     ['mul', 'add', False],\
     ['mul', 'mul', True],\
     ['mul', 'mul', False],\
     ['mul', None],\
     [None, 'add', True],\
     [None, 'add', False],\
     [None, 'mul', True],\
     [None, 'mul', False],\
     [None, None]]

    #print(flags)

    #flags = [['T','T'],['T','F'],['F','F'],['F','T']]       

    rmse_inner_flags = []

    for flag in flags:
        #print(flag)

        npa = []

        for trainv_ix, test_ix in cv_inner.split(data.index):

            npa.append([data.iloc[trainv_ix],data.iloc[test_ix],ETSModel,flag])

        #rmses_inner = []
        #rmses_inner
        #for n in npa:
            #r = evaluate_ets(n)
            #print(r)
            #rmses_inner.append(r)
        future = client.map(evaluate_ets, npa)        

        rmses_inner = []
        #flag_order = []
        #my intent was to capture future objects vs results and this gave me results

        for f in as_completed(future):
            #exclude errors
            if(f.status=="error"):
                #pass
                #this causes errors with [flag,rmse] being passed by evaluate_ets
                rmses_inner.append([flag,np.nan])
            else:
                rmses_inner.append(f.result())

        #I don't want to keep track of flag here
        #flag_order = [r[0] for r in results_rmses].copy()
        #print(rmses_inner)
        rmse_ = [r[1] for r in rmses_inner].copy()


        #print(rmse_)
        #print(flag)
        rmse_inner_flags.append([flag,np.nanmean(rmse_),np.nanstd(rmse_)])

    #rmse_scores = []
    #flag_rmses = []

    #for fn_ in range(0,len(flags)):
        #flag_ = flags[fn_]
        #rmse_scores.append([flag_rmses[fn_],rmses[fn_][1]])

    print("inner cv scores:")
    inner_rmse_df = pd.DataFrame(rmse_inner_flags,columns=['flags','mean','std'])
    #print("inner cv scores:",[r[rmses)
    #print("inner cv sdevs:",sdevs)

    scores = inner_rmse_df['mean'].values
    sdevs = inner_rmse_df['std'].values
    best_model = inner_rmse_df['flags'].values[np.argmin(scores)]

    print("best model:",best_model)
    print("best inner cv score, sdev (11 folds):",scores[np.argmin(scores)],sdevs[np.argmin(scores)])

    npa = []

    for trainv_ix, test_ix in cv_outer.split(data.index):

        npa.append([data.iloc[trainv_ix],data.iloc[test_ix],ETSModel,best_model])

    future = client.map(evaluate_ets, npa)
    results_rmses_outer = []

    #my intent was to capture future objects vs results and this gave me results
    for f in as_completed(future):
        #exclude errors
        if(f.status=="error"):
            #pass
            results_rmses_outer.append([np.nan])
        else:
            results_rmses_outer.append(f.result()) 

    test_score_mean = np.nanmean([r[1] for r in results_rmses_outer])
    test_score_std = np.nanstd([r[1] for r in results_rmses_outer])

    #test_score = [np.nanmean(rmse_outer_df['mean'].values),np.nanstd(rmse_outer_df['mean'].values)]
    print("test cv mean error, stdev (10 folds):",test_score_mean,test_score_std)

    #final_model=ETSModel(np.log(data),initialization_method='heuristic',seasonal=52,trend=F1,damped_trend=F2).fit()    


    #final_model=ETSModel(np.log(data),initialization_method='heuristic',seasonal=52,trend=F1,damped_trend=F2).fit()    
    flag = best_model
    if(len(flag)==2):
        if(flag[1] is None):
            final_model = ETSModel(np.log(data['Adj Close']),initialization_method='heuristic',seasonal=flag[0],trend=flag[1],dates=data.index).fit()
        else:
            final_model = ETSModel(np.log(data['Adj Close']),initialization_method='heuristic',seasonal=flag[0],seasonal_periods=52,trend=flag[1],dates=data.index).fit()
    else:
        final_model = ETSModel(np.log(data['Adj Close']),initialization_method='heuristic',seasonal=flag[0],seasonal_periods=52,trend=flag[1],damped_trend=flag[2],dates=data.index).fit()

    try:

        df_pred = np.exp(final_model.get_prediction(start=data.index[0].strftime('%Y-%m-%d'),end=(end + dt.timedelta(92)).strftime('%Y-%m-%d')).summary_frame())

    except:
        try:
            test = np.exp(pd.DataFrame(final_model.get_prediction(start=data.index[0].strftime('%Y-%m-%d'),end=(end + dt.timedelta(92)).strftime('%Y-%m-%d')).predicted_mean))
            test.columns = ['mean']
            df_pred = test
        except:
            df_pred = pd.DataFrame(np.repeat(np.nan,13),columns=['mean'])
            df_pred.index = [datetime.strptime(d, '%Y-%m-%d').date() for d in newDates] 

    #df_conf = np.exp(final_model.get_prediction(start=end,end=(end + dt.timedelta(92)).strftime('%Y-%m-%d')).pred_int(alpha = .05))
    #df_conf.columns = ['ci_lower','ci_upper']

    #final_model = ETSModel(np.log(data['Adj Close']),initialization_method='heuristic',seasonal=['add','mul','None'][0],seasonal_periods=52,trend=['add','mul','None'][0],damped_trend=['True','False'][0],dates=data.index).fit()
    #pred = final_model.get_prediction(start=end,end=(end + dt.timedelta(92)).strftime('%Y-%m-%d'))

    #final_model.get_prediction(start=end,end=(end + dt.timedelta(92)).strftime('%Y-%m-%d')).summary_frame()

    #df_pred = np.exp(pred.summary_frame(alpha=0.05))
    #df_pred = np.exp(pred.summary(alpha=0.05))

    s_date = df_pred.index[np.argmax(df_pred['mean'].values)].strftime('%Y-%m-%d')


    print("sell date:",s_date)
    e_return = np.array((np.max(df_pred['mean'].values)-metrics_df_['Adj Close'])/metrics_df_['Adj Close'])[0]
    p_metrics = pd.DataFrame(df_pred.iloc[np.argmax(df_pred['mean'].values)]).T
    #c_metrics = pd.DataFrame(df_conf.iloc[np.argmax(df_pred['mean'].values)]).T
    #print(c_metrics)
    print("expected return:",e_return)

    days_delta = (datetime.strptime(s_date, '%Y-%m-%d') - datetime.strptime(end.strftime('%Y-%m-%d'), '%Y-%m-%d')).days
    try:
        discounted_return = (1+e_return)**(1/days_delta)-1
    except:
         discounted_return = 0
    qtr_return = (1+discounted_return)**92

    print("discounted return:",discounted_return)

    print("quarter return:",qtr_return)

    try:
        #used to help filter at the end, not sdevs is chosen based on min score
        stop_loss_price = pd.DataFrame(np.mean(np.exp(final_model.get_prediction(start=end,end=(end + dt.timedelta(92)).strftime('%Y-%m-%d')).summary_frame())[['mean','pi_lower']],axis=1)).iloc[0].values[0]
        #decision_metrics.append([s,e_return,s_date,stop_loss_price,days_delta,discounted_return,qtr_return,test_score_mean,test_score_std,mean_revert_flag,p_metrics['pi_lower'].values[0],p_metrics['pi_upper'].values[0]])
        decision_metrics_df = pd.DataFrame([s,e_return,s_date,stop_loss_price,days_delta,discounted_return,qtr_return,test_score_mean,test_score_std,mean_revert_flag,p_metrics['pi_lower'].values[0],p_metrics['pi_upper'].values[0]])#,columns=)#,index=[s_])#.sort_values(by='disc_rtn',ascending=False,inplace=True)
        decision_metrics_df = decision_metrics_df.T
        decision_metrics_df.columns=['SYMBOL','exp_return','sell_date','stop_loss_price','days_delta','disc_rtn','qtr_rtn','outer_cv_avg_rmse_n11','outer_cv_std_n11','mean_revert_flag','pi_lower','pi_upper']
        decision_metrics_df.index = [symbol_]
    except:
        #used to help filter at the end, not sdevs is chosen based on min score
        stop_loss_price = df_pred[['mean']].iloc[0].values[0]
        #decision_metrics.append([s,e_return,s_date,stop_loss_price,days_delta,discounted_return,qtr_return,test_score_mean,test_score_std,mean_revert_flag,p_metrics['pi_lower'].values[0],p_metrics['pi_upper'].values[0]])
        decision_metrics_df = pd.DataFrame([s,e_return,s_date,stop_loss_price,days_delta,discounted_return,qtr_return,test_score_mean,test_score_std,mean_revert_flag])#,columns=)#,index=[s_])#.sort_values(by='disc_rtn',ascending=False,inplace=True)
        decision_metrics_df = decision_metrics_df.T
        decision_metrics_df.columns=['SYMBOL','exp_return','sell_date','stop_loss_price','days_delta','disc_rtn','qtr_rtn','outer_cv_avg_rmse_n11','outer_cv_std_n11','mean_revert_flag']
        decision_metrics_df.index = [symbol_]

    print("Outer CV Error / Price Ratio",decision_metrics_df['outer_cv_avg_rmse_n11']/screener_sorted.loc[symbol_]['Adj Close'])
    print(" Outer CV StdDev / Price Ratio",decision_metrics_df['outer_cv_std_n11']/screener_sorted.loc[symbol_]['Adj Close'])

    #display(decision_metrics_df.style.applymap(highlight_cells, color_if_false='yellow', color_if_true='#C6E2E9', subset=['error_price_ratio','mean_revert_flag','std_price_ratio'], threshold_=.75))

    try:

        temp_price = data
        temp_price.columns = ['mean']
        plt.plot(temp_price)
        plt.plot(df_pred)
        #plt.xticks([*data.tail(13).index.map(lambda t: t.strftime('%Y-%m-%d')),*newDates],rotation = 45,size=8)
        plt.xticks(rotation = 45,size=8)
        plt.legend(['actual','mean prediction','pi_lower','pi_upper'],loc=2)
        plt.show()

        plt.plot(temp_price.tail(13))
        plt.plot(df_pred.tail(26))
        #plt.xticks([*data.tail(13).index.map(lambda t: t.strftime('%Y-%m-%d')),*newDates],rotation = 45,size=8)
        plt.xticks(rotation = 45,size=8)
        plt.legend(['actual','mean prediction','pi_lower','pi_upper'],loc=2)
        plt.show()
    except:

        temp_price = data
        temp_price.columns = ['mean']
        plt.plot(temp_price)
        plt.plot(df_pred)
        #plt.xticks([*data.tail(13).index.map(lambda t: t.strftime('%Y-%m-%d')),*newDates],rotation = 45,size=8)
        plt.xticks(rotation = 45,size=8)
        plt.legend(['actual','mean prediction'],loc=2)
        plt.show()

        plt.plot(temp_price.tail(13))
        plt.plot(df_pred.tail(26))
        #plt.xticks([*data.tail(13).index.map(lambda t: t.strftime('%Y-%m-%d')),*newDates],rotation = 45,size=8)
        plt.xticks(rotation = 45,size=8)
        plt.legend(['actual','mean prediction'],loc=2)
        plt.show()
